# Sentence Classification

- 앞서 만든 영어 문장 긍/부정 분류기에 이어 함수를 간편화하고 다중 분류의 경우에도 문제를 해결할 수 있도록 만듦
- 이번 프로젝트에서는 한글 데이터인 NSMC 데이터와 한국어 감정 정보가 포함된 단발성 대화 데이터셋(AI Hub)을 이용함

- 해당 모델을 사용하기 위해 전처리 함수를 만들어 놓았으나 데이터에 어느 정도의 전처리가 필요
 - 전처리 함수 들어가면 중복 데이터, 결측치, id컬럼을 제거함
 - text 데이터의 컬럼은 sentences로 통일함
 - label의 데이터 타입이 str인 경우 str_label 컬럼을 만들어 줌


In [4]:
!pip3 install transformers

     |████████████████████████████████| 3.8 MB 4.3 MB/s 
     |████████████████████████████████| 6.5 MB 39.0 MB/s 
     |████████████████████████████████| 67 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 47.8 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import os
import json
import urllib.request

import pandas as pd
import numpy as np
import random
import time
import datetime

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader,TensorDataset,RandomSampler
from transformers import BertTokenizer,BertTokenizerFast
from transformers import get_linear_schedule_with_warmup
from transformers import BertConfig, BertForSequenceClassification, AdamW

from transformers import TextClassificationPipeline

from tqdm import tqdm
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.preprocessing import LabelEncoder

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [8]:
%cd /content/gdrive/My Drive/NLP

/content/gdrive/My Drive/NLP


In [9]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#  1. 데이터 전처리


In [14]:
def drop_na_and_duplicates(df):
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df = df.drop('id',axis=1) if 'id' in df.columns else df
    df.columns = ['sentences','str_label'] if df.iloc[:,-1].dtypes == 'O' else ['sentences','label']

    df['label'] = LabelEncoder().fit_transform(df['str_label'].tolist()) if 'str_label' in df.columns else df['label']

    return df

## 모델 불러오기 및 데이터 전처리 함수를 통해 전처리

### DATA 1. 네이버 영화 리뷰

- **긍정, 부정에 대한 감정 분석**

In [15]:
!git clone https://github.com/e9t/nsmc.git

train_data = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
train_data = drop_na_and_duplicates(train_data)

map_dict = {0 : '부정', 1 : '긍정'}

dev_data = pd.read_csv("nsmc/ratings_test.txt", sep='\t')
dev_data = drop_na_and_duplicates(dev_data)

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [16]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 149995 
dev data 개수 : 49997
{0: '부정', 1: '긍정'}


,sentences,label
100607,신선하고 흥미로운 초반 분위기에 비해 둔탁해진 후반전,0
3811,전편에 비해 한참 떨어진다..,0
94737,이 영화 보고 토할뻔 했어요... 조낸 재미 없어서요.,0
122624,평점올리게10점일단줌ㅋ존나무서움 제가공포영화는많이보는데 그해여름이게젤무서웟음,1
127667,1빠다ㅋㅋㅋ 별점테러라능 하핫,0


,sentences,label
35079,말조심하자는생각이드는영화,1
10829,OST 들으면 아직도 감동이와요..,1
17548,내용이 진지하고 어둡다 해서 점수 잘 줄 생각없다.,0
12921,마음 같아서는 1점이나 배우들 연기 때문에 4점 드립니다.,0
25398,피안도책을 봤는데 좀 B급 영화 티가 나는데 그럭저럭 볼만함,1


### DATA 2. 챗봇 데이터 

- **공포, 놀람, 중립, 분노, 슬픔, 행복, 혐오 ,공포 총 7개의 감정에 대한 감정 분석**

- 한국어 감정 정보가 포함된 단발성 대화 데이터셋

In [19]:
chatbot_data = pd.read_excel('datas/korean_chatbot.xlsx')
chatbot_data = drop_na_and_duplicates(chatbot_data)

map_dict = dict(zip(chatbot_data['str_label'].unique(),chatbot_data['label'].unique()))

train_data, dev_data= train_test_split(chatbot_data, test_size=0.2, random_state=42, shuffle=True)

In [20]:
print(f'train data 개수 : {len(train_data)} \ndev data 개수 : {len(dev_data)}')
print(map_dict)
display(train_data.sample(5))
display(dev_data.sample(5))

train data 개수 : 30822 
dev data 개수 : 7706
{'공포': 0, '놀람': 1, '분노': 2, '슬픔': 3, '중립': 4, '행복': 5, '혐오': 6}


,sentences,str_label,label
24591,내자동로터리에서 경찰들 미는 시위대들 정체가 뭐냐,중립,4
9177,김국진한테 반말은 아니지않나 훨씬선밴데 ㅋㅋ,놀람,1
21037,(≫≪) 미군 희생 여중생들의 죽음을 애도하며..,슬픔,3
17293,고민입니다 고민.,슬픔,3
1691,제가 계속 이 사장님을믿고 일해야할까요,공포,0


,sentences,str_label,label
35722,넘 유치하던데...연기도 오버,혐오,6
20094,어떻게 상처받은 눅희의 마음을 달래줄까요ㅜㅜ0,슬픔,3
22454,한국과 중국은 그런 단순한 관계가 아니다.,중립,4
3761,무서워요,공포,0
35994,아주 뻔뻔한 늙은 너구리구먼!!!!,혐오,6


# 2. 모델에 사용하기 위한 전처리

train data 전처리와 test data 전처리를 따로 분리해놓음

## 2-1. Data Preprocessing 

In [ ]:
def preprocess(df, batch_size=32, method = 'train'):
    sentence = [str(sentence) for sentence in  df['sentences'].tolist()]

    batch_input = tokenizer(sentence, padding=True, truncation=True)
    batch_input = {key : torch.tensor(value) for key, value in batch_input.items()}

    if method == 'train' or method == 'valid': 
        batch_input['labels'] = torch.tensor(df['label'].values)
        dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'], batch_input['labels'])
        if method == 'train':
            sampler = RandomSampler(dataset)
            dataloader = DataLoader(dataset, sampler=sampler, batch_size=batch_size)
        elif method == 'valid':
            dataloader = DataLoader(dataset, batch_size=batch_size)
            
    elif method == 'test':
        dataset = TensorDataset(batch_input['input_ids'], batch_input['attention_mask'], batch_input['token_type_ids'])
        dataloader = DataLoader(dataset, batch_size=batch_size)
        
    return dataloader

# 3. model 

## 1. model train

In [ ]:
 def train_one_epoch(optimizer, scheduler, dataloader):

     model.train()

     train_loss = 0.0

     for batchs in tqdm(dataloader):
         batch = tuple(b.to(device) for b in batchs)

         inputs = {
                 "input_ids": batch[0],
                 "attention_mask": batch[1],
                 "token_type_ids": batch[2],
                 'labels' : batch[3]
             }

         optimizer.zero_grad()

         output = model(**inputs)
         
         loss = output[0]
         
         loss.backward()

         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

         optimizer.step()
         scheduler.step()
        
         train_loss += loss.item()

     avg_train_loss = train_loss / len(dataloader)

     return avg_train_loss

## 2. model evaluation

In [ ]:
# 이거로 val_data와 test_data를 평가 가능
def evaluate_one_epoch(dataloader): 
    preds = []
    model.eval()

    eval_loss, eval_accuracy = 0, 0

    for batchs in tqdm(dataloader):
        batch = tuple(b.to(device) for b in batchs)
        inputs = {
                 "input_ids": batch[0],
                 "attention_mask": batch[1],
                 "token_type_ids": batch[2]
             }

        with torch.no_grad():
             output = model(**inputs)
                
        logits = output[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu()
        preds.append(logits)

    preds = torch.cat(preds, dim=0).argmax(dim=-1).tolist()

    return preds

## 3. model

In [ ]:
def sentiment_analysis_model(train_data, dev_data, lr=1e-4,epochs = 4, batch_size=32,bert='klue/bert-base', save=True, path='bert_tc'):
    gc.collect()
    torch.cuda.empty_cache()

    global tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(bert)

    global model, device 
    model = BertForSequenceClassification.from_pretrained(bert,num_labels=len(train_data.iloc[:,-1].value_counts()))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    train_dataloader  = preprocess(train_data,batch_size=batch_size, method='train')
    valid_dataloader  = preprocess(dev_data, batch_size=batch_size, method='valid')
    print('')
    print('Data Preprocessing Complete!')


    optimizer = Adam(model.parameters(),
                    lr = lr, # 학습률
                    eps = 1e-8)

    total_steps = len(train_dataloader) * epochs

    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = total_steps)

    print('')
    print('Model Training Start')
    print(f'Epochs : {epochs} / Learning Rate : {str(lr)} / Batch Size : {batch_size}')
    print('')

    loss = []
    acc = []
    for epoch in range(1,epochs+1):
        gc.collect()
        torch.cuda.empty_cache()
        print(f"epoch = {epoch}")

        train_loss = train_one_epoch(optimizer, scheduler, dataloader=train_dataloader)
        loss.append(train_loss)

        preds = evaluate_one_epoch(dataloader=valid_dataloader)

        val_acc = accuracy_score(dev_data['label'].tolist(),preds)
        acc.append(val_acc)
        
        print(f"Loss = {train_loss:.3f} / Accuracy = {val_acc:.3f}")
        print('')

        if save:
            model.save_pretrained(f'models/{path}')
            tokenizer.save_pretrained(f'models/{path}')
            print('Model Save')

    print('')
    print("Training Complete!")

    return {'loss':loss,'acc':acc}

In [ ]:
# bert = 'klue/bert-base'
# bert = 'beomi/kcbert-base'

score = sentiment_analysis_model(train_data, dev_data,lr=1e-5,batch_size=32 ,epochs = 4, bert = 'klue/bert-base', path='multi_class_test')

# 4. Test

In [ ]:
def Sentiment_Analysis(test_data, batch_size=32,bert='nsmc_test'):
    gc.collect()
    torch.cuda.empty_cache()

    global tokenizer
    tokenizer = BertTokenizer.from_pretrained(f'klue/bert-base')
    test_dataloader = preprocess(test_data, batch_size=batch_size, method='test')

    global model, device
    model = BertForSequenceClassification.from_pretrained(f'models/{bert}')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    pred_labels = evaluate_one_epoch(dataloader=test_dataloader)

    new_df = test_data[['document_1']].copy()
    new_df['pred_labels'] = pred_labels
    new_df['str_label'] = new_df['pred_labels'].apply(lambda x : map_dict[x])
    
    return  new_df

In [ ]:
df = Sentiment_Analysis(dev_data, batch_size=32,bert='nsmc_test')    
df.sample(5)

100%|██████████| 1563/1563 [03:17<00:00,  7.92it/s]


,document_1,pred_labels,str_label
46814,내 깅코를 ㅜ,1,긍정
48546,일본영화에다가 사극이라는 점을 제외해도 이정도면 잘 만든거다- _-,1,긍정
1324,시나리오 두번 다시 쓰지마라,0,부정
19874,생각보다 너무 슬프고 재밌다. 이 정도로 폐인 될 줄 몰랐음,1,긍정
26732,1회부터 계속 봐오던 통쾌한 복수를 예상했던 내가 바보,0,부정
